In [1]:
import numpy as np
import pandas as pd
import json
import math

In [2]:
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                       sep=',',
                        usecols=["item_id","n_clicks","views","clicks_views_ratio"])
item_metadata[:3]

,item_id,n_clicks,views,clicks_views_ratio
0,5001,3.0,99,0.030303
1,5002,6.0,68,0.088235
2,5003,6.0,90,0.066667


In [ ]:
train_set = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\data\\train.csv',
                       sep=',',
#                         skiprows=range(1,3000000),
#                       nrows=1000000,
                      index_col=0)
train_set[:3]

In [ ]:
train_set = train_set[train_set['action_type'] == 'clickout item']

In [ ]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_impressions_and_prices(df_in):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, 'impressions'] = df['impressions'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df['impressions'].str.len())
         for col in df.columns.drop('impressions')}
    )
    

    df_out.loc[:, 'impressions'] = np.concatenate(df['impressions'].values)
    df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out

In [ ]:
import time
start = time.time()
train_set = explode_impressions_and_prices(train_set)
print(time.time() - start)
train_set[:3]

In [ ]:
train_set = train_set[['reference', 'impressions']]
train_set[:3]

In [ ]:
start = time.time()
train_set = train_set.merge(item_metadata,
                           left_on='impressions',
                           right_on='item_id',
                           how='left')
train_set = train_set.drop(['impressions'], axis=1)
train_set = train_set.fillna(0)
# train_set.iloc[:,4:] = train_set.iloc[:,4:].astype('int8')
print(time.time() - start)
train_set["reference"] = train_set["reference"].astype(int)
train_set[:5]

In [ ]:
# train_set.iloc[:,:2] = train_set.iloc[:,:2].astype(int)
train_set['clicked'] = np.where(train_set['reference'] == train_set['item_id'], 1, 0)
train_set = train_set.drop(columns=['reference','item_id'])
train_set[:3]

In [ ]:
# NAUKA

In [ ]:
train_set['clicked'].value_counts()

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# out = out.dropna()
X = train_set.iloc[:, :-1]
y = train_set['clicked'].values

In [ ]:
from sklearn.preprocessing import StandardScaler, normalize
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
# X_train = normalize(X_train, norm='l2')
X_test = sc.transform(X_test)
# X_test = normalize(X_test, norm='l2')

In [ ]:
# from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import RandomUnderSampler
cc = RandomUnderSampler(random_state=0)
X_train, y_train = cc.fit_resample(X_train, y_train)

In [ ]:
# from imblearn.over_sampling import ADASYN
# adasyn = ADASYN(random_state=1, n_jobs=-1)
# X_train, y_train = adasyn.fit_resample(X_train, y_train)

In [ ]:
logreg = LogisticRegression(n_jobs=-1) #, class_weight={1.0:0.4, 0.0:0.6}
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
# THRESHOLD = 0.7
# y_pred = np.where(logreg.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))

print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())

print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

In [ ]:
# TEST

In [ ]:
test_set = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\data\\test.csv',
                       sep=',',
#                        nrows=1472418
                      skiprows=range(1,1472419)
                      )
mask = test_set["reference"].isnull() & (test_set["action_type"] == "clickout item")
test_set = test_set[mask]
test_set[:3]

In [ ]:
# test_set[1676230:]

In [ ]:
test_set = explode_impressions_and_prices(test_set)
test_set[:3]

In [ ]:
test_set = test_set[['impressions', 'number_of_impressions', 'step', 'user_id','session_id', 'timestamp']]
test_set[:3]

In [ ]:
test_set = test_set.merge(item_metadata,
                           left_on='impressions',
                           right_on='item_id',
                           how='left')

In [ ]:
test_set = test_set.drop(['impressions'], axis=1)
test_set = test_set.fillna(0)
test_set.iloc[:,8:] = test_set.iloc[:,8:].astype('int8')
test_set.iloc[:,6:7] = test_set.iloc[:,6:7].astype('int8')

test_set[:5]

In [ ]:
item_ids = test_set[['item_id','number_of_impressions', 'step', 'user_id','session_id','timestamp']]
test_set = test_set.drop(columns=['item_id', 'number_of_impressions', 'step', 'user_id','session_id','timestamp'])
test_set[:3]

In [ ]:
X_test = sc.transform(test_set.values)
X_test = normalize(X_test, norm='l2')

In [ ]:
results = logreg.predict_proba(X_test)

In [ ]:
df_results = pd.DataFrame(results)
df_results[['item_id', 'number_of_impressions', 'step', 'user_id','session_id','timestamp']] = item_ids
df_results = df_results[['item_id', 'number_of_impressions', 'step', 'user_id','session_id', 'timestamp', 0, 1]]
df_results['item_id'] = df_results['item_id'].astype(int)
df_results[:3]

In [ ]:
def ddd(df):
    return df[0].max()

In [ ]:
df = pd.DataFrame(df_results.groupby(['item_id','number_of_impressions','step','user_id','session_id','timestamp']).apply(ddd)).reset_index()

In [ ]:
df = df.sort_values(['user_id','session_id','timestamp',0], ascending=[True, True, True, False])
df = df.drop(columns=['number_of_impressions'])
df = df[['user_id', 'session_id','timestamp','step',0]]
df = df.reset_index()
df = df.rename(columns={'index':'item_id'})
df[:5]

In [ ]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    
    return df_out

In [ ]:
df['item_id'] = df['item_id'].astype(str)
df = group_concat(df, ["user_id", "session_id", "timestamp", "step"], 'item_id')
df = df.rename(columns={'item_id':'item_recommendations'})
df[:5]

In [ ]:
df.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\submission_ML2.csv", sep=',', index=False)

In [ ]:
df1 = df.copy()

In [ ]:
df1 = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\submission_ML1.csv',
                       sep=',')
df1[:3]

In [ ]:
df2 = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\submission_ML2.csv',
                       sep=',')
df2[:3]

In [ ]:
print(df1.shape)
print(df2.shape)

In [ ]:
df1 = pd.concat([df1, df2])
df1.shape

In [ ]:
df1.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\submission_ML.csv", sep=',', index=False)